In [26]:
!pip install transformers datasets evaluate > /dev/null

In [2]:
import evaluate

#metric

accuracy = evaluate.load("accuracy")

In [3]:
meteor = evaluate.load("meteor",module_type='metric')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
ex_match = evaluate.load("exact_match",module_type='comparison')

In [6]:
label_dist = evaluate.load("label_distribution",
                           module_type='measurement')

In [7]:
#get the available evaluation functions inside a module

metrics_list = evaluate.list_evaluation_modules(
  module_type="metric",
  include_community=False,
  with_details=True)

In [8]:
len(metrics_list)

53

In [10]:
metrics_list[0]

{'name': 'precision', 'type': 'metric', 'community': False, 'likes': 0}

In [11]:
comparer_list = evaluate.list_evaluation_modules(
  module_type="comparison",
  include_community=False,
  with_details=True)

In [12]:
comparer_list[0]

{'name': 'mcnemar', 'type': 'comparison', 'community': False, 'likes': 1}

In [13]:
measure_list = evaluate.list_evaluation_modules(
  module_type="measurement",
  include_community=False,
  with_details=True)

In [14]:
measure_list[0]

{'name': 'word_length', 'type': 'measurement', 'community': False, 'likes': 0}

In [ ]:
#attributes / methods
dir(accuracy)

**description**	A short description of the evaluation module.

**citation**	A BibTex string for citation when available.

**features**	A Features object defining the input format.

**inputs_description**	This is equivalent to the modules docstring.

In [18]:
print(accuracy.description)


Accuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
 Where:
TP: True positive
TN: True negative
FP: False positive
FN: False negative



In [19]:
accuracy.features

{'predictions': Value(dtype='int32', id=None),
 'references': Value(dtype='int32', id=None)}

Note that features always describe the type of a single input element. In general we will add lists of elements so you can always think of a list around the types in features. Evaluate accepts various input formats (Python lists, NumPy arrays, PyTorch tensors, etc.) and converts them to an appropriate format for storage and computation.

## Compute metric

- compute

- add

- add_batch 

-

In [21]:
# pass inputs as lists. You will see the results are interpretable

accuracy.compute(references=[1,1,1,0],
                 predictions=[1,0,0,1])

{'accuracy': 0.25}

In [22]:
for ref, pred in zip([0,1,1,1],[1,1,1,0]):
  accuracy.add(references=ref, predictions=pred)

accuracy.compute()

{'accuracy': 0.5}

In [23]:
for refs, preds in zip([[0,1],[0,1]], [[1,0],[0,1]]):
    accuracy.add_batch(references=refs, predictions=preds)

accuracy.compute()

{'accuracy': 0.5}

In [24]:
clf_metrics = evaluate.combine(["accuracy", 
                                "f1", 
                                "precision", 
                                "recall"])

In [25]:
for refs, preds in zip([[0,1],[0,1]], [[1,0],[0,1]]):
    clf_metrics.add_batch(references=refs, predictions=preds)

clf_metrics.compute()

{'accuracy': 0.5, 'f1': 0.5, 'precision': 0.5, 'recall': 0.5}

In [28]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate

pipe = pipeline("text-classification", 
                model="lvwerra/distilbert-imdb")

In [29]:
data = load_dataset("imdb", 
                    split="test").shuffle().select(range(1000))

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


In [31]:
task_evaluator = evaluator("text-classification")

In [ ]:
results = task_evaluator.compute(model_or_pipeline=pipe, 
                                 data=data, 
                                 metric=clf_metrics,
                       label_mapping={"NEGATIVE": 0, "POSITIVE": 1},)

In [ ]:
print(results)